# **The Basic Concept of Difference Form Standard Deviation**
By: Iyas Yustira, Dena Supriyanto

Reference: [Ravi, J. & Selvapandian, Dickson & Akila, R & Sathya, K. (2019). An Optimal Solution for Transportation problem-DFSD 1. Journal of Computational Mathematica. 3. 43. 10.26524/cm46.](https://www.researchgate.net/publication/333894284_An_Optimal_Solution_for_Transportation_problem-DFSD_1)

## **DFSD Method**

**Row penalty**

for $i=1,2,3,\cdots,m$

$$\sigma_i = \sqrt{\frac{\sum_j^n (x_j - \bar{x_i})^2}{n}}$$

**Column penalty**

for $j=1,2,3,\cdots,n$

$$\sigma_j = \sqrt{\frac{\sum_i^m (x_i - \bar{x_j})^2}{m}}$$

Where, m is the number of rows, n is the number of columns and x is the corresponding row/column values

## **DFSD Algorithm**

The result of DFSD method the smallest unit cost element in the row/column (cell) from the immediate next smallest unit cost element in the same row/column is determining a penalty measure for the target
row/column.

i). This step includes the following sub-steps:

* Identify the row or the column that includes the largest penalty.
* Break ties arbitrarily
* As much as possible, the lowest cost row/column (cell) in the row or column should be allocated with the highest difference.
* Adjust the supply and demand, and then cross out the satisfied row or column.
* If a row and column are satisfied simultaneously, then only one of them is crossed out, as well the remains rows or columns are assigned to supply as zero (demand).


ii). Finally, the result should be computed as follows:

* If a row or a column is assigned as zero supply, or demand remains uncrossed out, then stop the process.
* If one row/column with positive supply (demand) remains uncrossed out, then determine the basic variables in the row/column by the lowest cost method, and then stop.
* If all the uncrossed out rows and columns have (remaining) zero supply and demand then determine the zero basic variables by the lowest cost method and stop.
* Otherwise, go to step (i).

## **Build The ALgorithm**

Let's consider this tranportation problem. Find the Optimal solution for the given T.P.

<img src="img/table df.png" width=350px>

**Step 1**

Prepare the table.

In [1]:
import pandas as pd
import numpy as np

d = {'Index': ['O1', 'O2', 'O3', 'Demand'], 
     'D1': [4, 2, 7, 7],
     'D2': [7, 1, 8, 10],
     'D3': [8, 10, 3, 6],
     'D4': [3, 12, 4, 12],
     'Supply': [10, 15, 10, 0]}
data = pd.DataFrame(data=d)
data.set_index('Index', inplace=True)
data

,D1,D2,D3,D4,Supply
Index,,,,,
O1,4,7,8,3,10
O2,2,1,10,12,15
O3,7,8,3,4,10
Demand,7,10,6,12,0


**Step 2**

Check the balanced data using function below. 

For more detail about is_balanced() function see the **Step 2** in <a href="Basic Concept ZU.ipynb">this notebook</a>.

In [2]:
def is_balanced(data):
    sum_a = data['Supply'].sum()
    sum_b = data.loc['Demand'].sum()

    if sum_a < sum_b:
        print('Supply < Demand : {:d} < {:d}'.format(sum_a, sum_b))
        data_T = data.T
        dm = data_T.pop('Demand')
        data_T['Dum'] = np.zeros(len(dm)).astype(int)
        data_T['Demand'] = dm
        data = data_T.T
        data.loc['Dum', 'Supply'] = sum_b - sum_a
    elif sum_a > sum_b:
        print('Supply > Demand : {:d} > {:d}'.format(sum_a, sum_b))
        Supply = data.pop('Supply').reset_index()
        data['Dum'] = np.zeros(len(data.index)).astype(int)
        data['Supply'] = Supply.Supply.to_numpy()
        data.loc['Demand', 'Dum'] = sum_a - sum_b
    else: 
        print('Supply = Demand : {:d}'.format(sum_a))
        pass
    
    return data

In [3]:
data = is_balanced(data)

Supply = Demand : 35


**Step 3**

Calculate the penalty row/column using standard deviation equation. If the cell just one row/column then the penalty is itself.

In [4]:
def penalty(data):
    data['Penalty'] = np.zeros(data.shape[0]).astype(int)
    data.loc['Penalty'] = np.zeros(data.shape[1]).astype(int)
    
    m = data.index[:-2]
    n = data.columns[:-2]
    
    # row penalty
    for i in m:
        if len(n) == 1:
            data.loc[i, 'Penalty'] = data.loc[i][:-2][0]
        else:
            data.loc[i, 'Penalty'] = np.round(data.loc[i][:-2].to_numpy().std(), 2)
    
    # column penalty
    for j in n:
        if len(m) == 1:
            data.loc['Penalty', j] = data[j][:-2][0]
        else:
            data.loc['Penalty', j] = np.round(data[j][:-2].to_numpy().std(), 2)
    
    return data

In [5]:
data = penalty(data)
data

,D1,D2,D3,D4,Supply,Penalty
Index,,,,,,
O1,4.00,7.00,8.00,3.00,10,2.06
O2,2.00,1.00,10.00,12.00,15,4.82
O3,7.00,8.00,3.00,4.00,10,2.06
Demand,7.00,10.00,6.00,12.00,0,0.00
Penalty,2.05,3.09,2.94,4.03,0,0.00


**Step 4**

Identify the row/column that includes the largest penalty for cost allocation. If there is tie between the largest penalty row/column, take the greatest least cost $(x_{ij})$ between them.

In [6]:
def cell_allocation(data):
    pr_max = data['Penalty'][:-2].max()
    pc_max = data.loc['Penalty'][:-2].max()
    
    m = data[data['Penalty'] == pr_max].index
    n = data.loc['Penalty'][data.loc['Penalty'] == pc_max].index
    
    glc = {}
    if pr_max == pc_max:
        for i in m:
            glc[data.loc[i][:-2].min()] = i
        for j in n:
            glc[data[j][:-2].min()] = j
        rc = glc[max(glc)]
        if rc in m:
            ri = rc
            cj = data.loc[ri][:-2][data.loc[ri][:-2] == max(glc)].index[0]
        else:
            cj = rc
            ri = data[cj][:-2][data[cj][:-2] == max(glc)].index[0]
    elif pr_max > pc_max:
        for i in m:
            glc[data.loc[i][:-2].min()] = i
        ri = glc[max(glc)]
        cj = data.loc[ri][:-2][data.loc[ri][:-2] == max(glc)].index[0]
    else:
        for j in n:
            glc[data[j][:-2].min()] = j
        cj = glc[max(glc)]
        ri = data[cj][:-2][data[cj][:-2] == max(glc)].index[0]
        
    print('Allocation to {:s} and {:s}'.format(ri, cj))
    return ri, cj

In [7]:
ri, cj = cell_allocation(data)

Allocation to O2 and D2


**Step 5**

Identify the minimum cost (Supply/Demand) based on the cell above and allocate to that cell. Then, crossed out the row/column if the corresponding row/column is zero.

Similar, with the **step 5** in <a href="Basic Concept ZU.ipynb">this notebook</a>.

In [8]:
def cost_allocation(ri, cj, data):
    ai = data.loc[ri, 'Supply']
    bi = data.loc['Demand', cj]
    
    if ai > bi:
        val = (bi * data.loc[ri, cj]).astype(int)
        cost.append(val)
        data.loc[ri, 'Supply'] = ai - bi
        data.drop(cj, axis=1, inplace=True)
    else:
        val = (ai * data.loc[ri, cj]).astype(int)
        cost.append(val)
        data.loc['Demand', cj] = bi - ai
        data.drop(ri, axis=0, inplace=True)
    print('Cost: {:d}'.format(val))
    return data

In [9]:
cost = []
data = cost_allocation(ri, cj, data)

Cost: 10


For the first iteration, we get the cost allocation is 10 with allocation to O2 and D2. 

### **Loop iteration**

In [10]:
i = 1
while True:
    i += 1
    print('-'*15)
    print(' Iteration: {:d}'.format(i))
    print('-'*15, '\n')
    data = penalty(data)
    print(data, '\n')
    m = data.index[:-2]
    n = data.columns[:-2]
    if len(m) == 1 and len(n) == 1:
        ri = m[0]
        cj = n[0]
        ai = data.loc[ri, 'Supply']
        val = (ai * data.loc[ri, cj]).astype(int)
        cost.append(val)
        print('Allocation to {:s} and {:s}'.format(ri, cj))
        print('Cost: {:d}'.format(val))
        break
    else:
        ri, cj = cell_allocation(data)
        data = cost_allocation(ri, cj, data)
    print()
print()
print("-"*23)
print('  Total Cost: {:d}'.format(sum(cost)))
print('-'*23, '\n')

---------------
 Iteration: 2
--------------- 

           D1     D3     D4  Supply  Penalty
Index                                       
O1       4.00   8.00   3.00      10     2.16
O2       2.00  10.00  12.00       5     4.32
O3       7.00   3.00   4.00      10     1.70
Demand   7.00   6.00  12.00       0     0.00
Penalty  2.05   2.94   4.03       0     0.00 

Allocation to O2 and D1
Cost: 10

---------------
 Iteration: 3
--------------- 

          D1   D3    D4  Supply  Penalty
Index                                   
O1       4.0  8.0   3.0      10     2.16
O3       7.0  3.0   4.0      10     1.70
Demand   2.0  6.0  12.0       0     0.00
Penalty  1.5  2.5   0.5       0     0.00 

Allocation to O3 and D3
Cost: 18

---------------
 Iteration: 4
--------------- 

          D1    D4  Supply  Penalty
Index                              
O1       4.0   3.0      10      0.5
O3       7.0   4.0       4      1.5
Demand   2.0  12.0       0      0.0
Penalty  1.5   0.5       0      0.0 

Alloc